In [1]:
import numpy as np
from sympy import *
from sympy.matrices import Matrix

First we create the symbols to be used in our Denavit-Hardenberg Calculations

In [2]:
# Create symbols
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') # joint angle theta_i
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8') # link offset
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7') # link length
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7') # twist angle

Then create DH parameters

In [3]:
### KUKA KR210 ###
# DH parameters
s = {alpha0:     0,  a0:      0, d1:  0.75,  
     alpha1: -pi/2,  a1:   0.35, d2:     0, q2: q2-pi/2, 
     alpha2:     0,  a2:   1.25, d3:     0,       
     alpha3: -pi/2,  a3: -0.054, d4:  1.50,
     alpha4:  pi/2,  a4:      0, d5:     0,
     alpha5: -pi/2,  a5:      0, d6:     0,
     alpha6:     0,  a6:      0, d7: 0.303, q7: 0}

Next we define the transformation matrices from the base frame to the gripper frame.

In [4]:
# Define Modified DH Transformation matrix
def build_transformation_matrix( theta, alpha, a, d ):
    T = Matrix([[            cos(theta),           -sin(theta),           0,             a ],
                [ sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -sin(alpha)*d ],
                [ sin(theta)*sin(alpha), cos(theta)*sin(alpha),  cos(alpha),  cos(alpha)*d ],
                [                     0,                     0,           0,             1 ]]) 
    T = T.subs(s)
       
    return(T)

# Modified DH params
T0_1 = build_transformation_matrix( q1, alpha0, a0, d1 )
T1_2 = build_transformation_matrix( q2, alpha1, a1, d2 )
T2_3 = build_transformation_matrix( q3, alpha2, a2, d3 )
T3_4 = build_transformation_matrix( q4, alpha3, a3, d4 )
T4_5 = build_transformation_matrix( q5, alpha4, a4, d5 )
T5_6 = build_transformation_matrix( q6, alpha5, a5, d6 )
T6_G = build_transformation_matrix( q7, alpha6, a6, d7 )

print("T0_1", T0_1)
print("T1_2", T1_2)
print("T2_3", T2_3)
print("T3_4", T3_4)
print("T4_5", T4_5)
print("T5_6", T5_6)
print("T6_G", T6_G)

T0_1 Matrix([[cos(q1), -sin(q1), 0, 0], [sin(q1), cos(q1), 0, 0], [0, 0, 1, 0.750000000000000], [0, 0, 0, 1]])
T1_2 Matrix([[sin(q2), cos(q2), 0, 0.350000000000000], [0, 0, 1, 0], [cos(q2), -sin(q2), 0, 0], [0, 0, 0, 1]])
T2_3 Matrix([[cos(q3), -sin(q3), 0, 1.25000000000000], [sin(q3), cos(q3), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
T3_4 Matrix([[cos(q4), -sin(q4), 0, -0.0540000000000000], [0, 0, 1, 1.50000000000000], [-sin(q4), -cos(q4), 0, 0], [0, 0, 0, 1]])
T4_5 Matrix([[cos(q5), -sin(q5), 0, 0], [0, 0, -1, 0], [sin(q5), cos(q5), 0, 0], [0, 0, 0, 1]])
T5_6 Matrix([[cos(q6), -sin(q6), 0, 0], [0, 0, 1, 0], [-sin(q6), -cos(q6), 0, 0], [0, 0, 0, 1]])
T6_G Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0.303000000000000], [0, 0, 0, 1]])


Because of the differences between the Denavit-Hardenberg convention and the definition in the URDF-File, we must apply a correctional transform to the last frame to calculate the position in the world-frame. Therefore we first apply a body-fixed rotation around z, then a body-fixed rotation around y.

R_corr is the composition of those two rotations.

In [5]:
#Correct the Gripper
R_z =  Matrix([[     cos(np.pi), -sin(np.pi),             0, 0],
               [     sin(np.pi),  cos(np.pi),             0, 0],
               [              0,           0,             1, 0],
               [              0,           0,             0, 1]])

R_y =  Matrix([[  cos(-np.pi/2),           0, sin(-np.pi/2), 0],
               [              0,           1,             0, 0],
               [ -sin(-np.pi/2),           0, cos(-np.pi/2), 0],
               [              0,           0,             0, 1]])

R_corr = simplify(R_z * R_y)

Now we create individual transformation matrices from base_link to the current link.

In [6]:
# Create individual transformation matrices
T0_2 = simplify(T0_1 * T1_2) # base_link to link_2
T0_3 = simplify(T0_2 * T2_3) # base_link to link_3
T0_4 = simplify(T0_3 * T3_4) # base_link to link_4
T0_5 = simplify(T0_4 * T4_5) # base_link to link_5
T0_6 = simplify(T0_5 * T5_6) # base_link to link_6
T0_G = simplify(T0_6 * T6_G) # base_link to gripper_link
T0_G_corr = simplify(T0_G * R_corr) # Total HT between base_link and gripper_link with orientation correction applied



We extract the end-effector position and orient from the request in ROS. In this notebook, those values are given from the test-dataset. From the given roll, pitch and yaw angles we can calculate a transformation matrix Rrpy which describes the current position and orientation.

Rrpy is a Matrix calculated from a rotation Rz \* Ry \* Rx

In [7]:
# Build a rotation matrix from the Roll, Pitch and Yaw angles
def get_wrist_rot_matrix(roll, pitch, yaw):
    
    # Rrpy = Rz * Ry * Rx
    R_z =  Matrix([[     cos(yaw), -sin(yaw),             0],
                   [     sin(yaw),  cos(yaw),             0],
                   [              0,           0,             1]])

    R_y =  Matrix([[  cos(pitch),           0, sin(pitch)],
                   [              0,           1,             0],
                   [ -sin(pitch),           0, cos(pitch)]])
    
    R_x = Matrix([[ 1,              0,        0],
                  [ 0,        cos(roll), -sin(roll)],
                  [ 0,        sin(roll),  cos(roll)]])
     
    Rrpy = simplify ( R_z * R_y * R_x )
    
    return Rrpy

Rrpy = get_wrist_rot_matrix(q1, q2, q3)
print(Rrpy)

Matrix([
[cos(q2)*cos(q3), sin(q1)*sin(q2)*cos(q3) - sin(q3)*cos(q1),  sin(q1)*sin(q3) + sin(q2)*cos(q1)*cos(q3)],
[sin(q3)*cos(q2), sin(q1)*sin(q2)*sin(q3) + cos(q1)*cos(q3), -sin(q1)*cos(q3) + sin(q2)*sin(q3)*cos(q1)],
[       -sin(q2),                           sin(q1)*cos(q2),                            cos(q1)*cos(q2)]])


With Rrpy we can find the location of the wrist center (WC) relative to the base frame. 

l (column 1 of Rrpy) is the vector along gripper_link x-axis.

The end_effector_length is the distance between joint 5 and the tip of the gripper fingers. 

Finally we extend Rrpy with the given positions of the gripper to create a 4x4 transformation matrix we can later use in calculations.

In [8]:
def get_wrist_pos(px, py, pz, Rrpy):
           
    #end_effector_length = 0.453
    end_effector_length = 0.303
    
    
    lx = Rrpy[ 0, 0 ]
    ly = Rrpy[ 1, 0 ]
    lz = Rrpy[ 2, 0 ]

    d6 = 0

    # Calculate Wrist Center
    wx = px - ( end_effector_length + d6 ) * lx
    wy = py - ( end_effector_length + d6 ) * ly
    wz = pz - ( end_effector_length + d6 ) * lz
    
    # Modify Rrpy to take in the transformation from gripper
    Rrpy = Rrpy.row_join(Matrix([[px],[py],[pz]]))
    Rrpy = Rrpy.col_join(Matrix([[0,0,0,1]]))

    return wx, wy, wz, Rrpy

wx, wy, wz, Rrpy = get_wrist_pos( 0, 0, 0, Rrpy)
print(Rrpy)

Matrix([
[cos(q2)*cos(q3), sin(q1)*sin(q2)*cos(q3) - sin(q3)*cos(q1),  sin(q1)*sin(q3) + sin(q2)*cos(q1)*cos(q3), 0],
[sin(q3)*cos(q2), sin(q1)*sin(q2)*sin(q3) + cos(q1)*cos(q3), -sin(q1)*cos(q3) + sin(q2)*sin(q3)*cos(q1), 0],
[       -sin(q2),                           sin(q1)*cos(q2),                            cos(q1)*cos(q2), 0],
[              0,                                         0,                                          0, 1]])


Next we find joint variables, theta1, theta2 and theta3.

For Theta 1 we can just take the x and y coordinates we calculated for the wrist center and derive theta1 from tha atan2-function.
![Deriving Theta 1](writeup_images/theta1.png)


Theta 2 is a bit trickier. First, we calculate the wrist center coordinates to match the origin of Frame O_1. Therefor we substract a1 as a horizontal offset from x1 and d1 as a vertical offset from z1. With those new coordinates, we can calculate the distance from Joint2 to Joint5: L25. With the fixed length a3 and d4 we can calculate the distance from Joint3 to Joint5: L35.

Now width the arm-length a2, L25 and L35 we have our triangle and are ready to perform trigonometric calculations. First we calculate the angle *beta2* between a2 and L25 with atan2. For *beta1* we use the cosine-law. As theta2 has a minus 90° Offset, we now have to subtract those two angles from 90°.
![Deriving Theta 2](writeup_images/theta2.png)

For theta3 we can use the already calculated L35 length along with a3 and d4 to calculate a fixed offset angle *beta4*. Additionally, we use the cosine law again to calculate the angle between a2 and L35. To derive theta3 we have to subtract beta3 and beta 4 from the 90° between a2 and d4.
![Deriving Theta 3](writeup_images/theta3.png)

In [9]:
# Calculate joint angles using Geometric IK method
def get_theta_123 (wx, wy, wz):
    
    theta1 = atan2(wy, wx).evalf()
    
    a_1 = 0.35
    a_2 = 1.25
    d_1 = 0.75
    d_4 = 1.50
    a_3 = -0.054
    
  
    r = sqrt( wx**2 + wy**2 ) - a_1
    z_c = wz - d_1 # Subtract d1 as vertical offset from robot base

    
    
    L_25 = sqrt( r**2 + z_c**2 )
    L_35 = sqrt( a_3**2 + d_4**2 )
      
    beta_2 = atan2( z_c, r )
    cos_beta_1 = ( ( L_35**2 - L_25**2 - a_2**2 ) / ( - 2 * L_25 * a_2 ) )
    beta_1 = atan2(  sqrt( 1 - cos_beta_1**2 ),  cos_beta_1 ) 
    # this would be the flipped over position, but we will not reach it in this project
    #beta_12 = atan2(  - sqrt( 1 - cos_beta_1**2 ), cos_beta_1 ) 

    theta2 = np.pi/2 - beta_1 - beta_2
    
    beta_4 = atan2( -a_3, d_4 )
    cos_beta_3 = ( L_25**2 - a_2**2 - L_35**2 ) / ( -2 * a_2 * L_35 )
    beta_3 = atan2( sqrt( 1 - cos_beta_3**2 ), cos_beta_3 )
    # this would be the flipped over position, but we will not reach it in this project
    #beta_32 = atan2( - sqrt( 1 - cos_theta_3**2 ), cos_theta_3 )
    
    theta3 = np.pi/2 - beta_3 - beta_4
        
    return theta1.evalf(), theta2.evalf(), theta3.evalf()

theta1, theta2, theta3 = get_theta_123( 1.8945, -1.4430, 1.6937 )
print(theta1, theta2, theta3)

-0.650933574600653 0.448192818182759 -0.362059820437478


Once the first three joint variables are known, we calculate R0_3 via application of homogeneous transforms up to the WC. Then we calculate the R3_6_var Matrix, which contains the remaining unknown thetas 4-6.

By multiplying the inverse of R0_3 with Rrpy we can retrieve R3_6_eval, which contains numerical values. This is important for the next step, where we use those two matrices to generate equations to solve for the thetas.

In [10]:
def transform_to_wc(theta1, theta2, theta3, Rrpy):
 
    R0_3 = simplify( T0_1 * T1_2 * T2_3 )
    R0_3 = R0_3.evalf(subs={q1: theta1, q2: theta2, q3: theta3})
    R3_6_eval = R0_3.inv() * Rrpy * R_corr
    R3_6_var = simplify( T3_4 * T4_5 * T5_6 )
    
    return R3_6_var, R3_6_eval        

To get the composite rotation matrix R3_6 we multiply the precalculated T3_4, T4_5 and T5_6.
![Deriving R3_6](writeup_images/R3_6.png)

In [11]:
R3_6_var, R3_6_eval = transform_to_wc( theta1, theta2, theta3, Rrpy )
print("R3_6:", R3_6_var)

R3_6: Matrix([
[-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -sin(q5)*cos(q4), -0.054],
[                           sin(q5)*cos(q6),                           -sin(q5)*sin(q6),          cos(q5),    1.5],
[-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6),  sin(q4)*sin(q5),      0],
[                                         0,                                          0,                0,      1]])


From this matrix we choose r_ij to calculate our angles. Those are the r_ji I choose for my calculations:

In [12]:
print("r12 = ", R3_6_var[0,1])
print("r13 = ", R3_6_var[0,2])
print("r21 = ", R3_6_var[1,0])
print("r22 = ", R3_6_var[1,1])
print("r23 = ", R3_6_var[1,2])
print("r32 = ", R3_6_var[2,1])
print("r33 = ", R3_6_var[2,2])

r12 =  -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5)
r13 =  -sin(q5)*cos(q4)
r21 =  sin(q5)*cos(q6)
r22 =  -sin(q5)*sin(q6)
r23 =  cos(q5)
r32 =  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6)
r33 =  sin(q4)*sin(q5)


Now we find a set of Euler angles corresponding to the rotation matrix.

We start with Theta 5 as r23 contains an isolated cos(q5). to get sin(q5) we calculate sqrt(r13^2 + r33^2), as they contain sin(q4) and cos(q4). Those combined equate to 1 and only leave sin(q5) remaining.
![Deriving Theta 5](writeup_images/theta5.png)

Now that we have q5 calculated, we can choose r_ij containing only q4 and q5 to calculate theta4.
![Deriving Theta 4](writeup_images/theta4.png)

And we can choose r_ij containing only q6 and q5 to calculate theta6.
![Deriving Theta 6](writeup_images/theta6.png)

There is just one problem left: if q5 = 0, we cannot use r33 and r13 to calculate theta4 and r22 and r21 to calculate theta6. In this case I set theta5 to 0, theta6 to its last known value and calculate theta4 using r12 and r32

In [13]:
last_t4 = 0.1
last_t5 = 0.1
last_t6 = 0.1

def get_theta_456(R3_6_var, R3_6_eval):
    r12 = R3_6_eval[0,1]
    r13 = R3_6_eval[0,2]
    r21 = R3_6_eval[1,0]
    r22 = R3_6_eval[1,1]
    r23 = R3_6_eval[1,2]
    r32 = R3_6_eval[2,1]
    r33 = R3_6_eval[2,2]

    # when theta5 is 0 we encounter a wrist singularity and cannot obtain theta4 and theta6 from r33/r13 r22/r21
    # in this case we set theta6 to its last value, theta 5 to zero and calculate theta4 in correspondence to theta6
    # Hint from @gwwang in slack channel
    if np.abs(r23) is not 1:
            sin_q5 = sqrt(r13**2 + r33**2).evalf()
            theta5 = atan2( sin_q5, r23 ).evalf()
            if( sin_q5 < 0 ):
                theta4 = atan2( -r33,  r13).evalf()
                theta6 = atan2(  r22, -r21).evalf()
            else:
                theta4 = atan2(  r33, -r13).evalf()
                theta6 = atan2( -r22,  r21).evalf()
    else:
        theta6 = self.last_t6
        if ( r23 == 1  ):  
            theta5 = 0 
            theta4 = -theta6 + atan2( -r12,  -r32).evalf()
        else:
            theta5 = 0 
            theta4 = theta6 + atan2( r12,  -r32).evalf()
            
        last_t4 = theta4
        last_t5 = theta5
        last_t6 = theta6
            
    return theta4, theta5, theta6   

Now that we have all our values calculated, lets see what we can derive from the test-data.
This is some test-input data for verifying the Inverse Kinematics using Forward Kinematics.

In [14]:
test_set_1 = { "xpos" : 2.16135, "ypos" : -1.42635, "zpos" : 1.5511, 
               "roll" : 1.6544,  "pitch" : 0.4899, "yaw" : 0.0624, 
               "theta1_rviz" : -0.65, "theta2_rviz" : -0.65, "theta3_rviz" : -0.65, 
               "theta4_rviz" : 0.95, "theta5_rviz" : 0.79, "theta6_rviz" : 0.49,
               "wcx_rviz" : 1.89451, "wcy_rviz" : -1.44302, "wcz_rviz" : 1.6937,}

# Choose current set of test data
current_test_set = test_set_1

In [15]:
Rrpy                                = get_wrist_rot_matrix(current_test_set["roll"], current_test_set["pitch"], current_test_set["yaw"])
test_wx,  test_wy,    test_wz, Rrpy = get_wrist_pos(current_test_set["xpos"], current_test_set["ypos"], current_test_set["zpos"], Rrpy)
theta1,   theta2,     theta3        = get_theta_123( test_wx, test_wy, test_wz )
R3_6_var, R3_6_eval                 = transform_to_wc( theta1, theta2, theta3, Rrpy )
theta4,   theta5,     theta6        = get_theta_456( R3_6_var, R3_6_eval )


And now for we compare the calculated values against expected values from the sim

In [16]:
print("---------------- input data ----------------")
print("px, py, pz: %7.4f  %7.4f  %7.4f"%  ( current_test_set["xpos"], current_test_set["ypos"], current_test_set["zpos"] ) )
print("r , p , y : %7.4f  %7.4f  %7.4f"%  ( current_test_set["roll"], current_test_set["pitch"], current_test_set["yaw"]) )
print("")
print("--------------- wrist center ---------------")
print("WC_x: %7.4f"% test_wx, "expected: %7.4f"% current_test_set["wcx_rviz"])
print("WC_y: %7.4f"% test_wy, "expected: %7.4f"% current_test_set["wcy_rviz"])
print("WC_z: %7.4f"% test_wz, "expected: %7.4f"% current_test_set["wcz_rviz"],"\n")
print("--------------- joint angles ---------------")
print("theta 1: %7.4f"% theta1, "expected: %7.4f"% current_test_set["theta1_rviz"])
print("theta 2: %7.4f"% theta2, "expected: %7.4f"% current_test_set["theta2_rviz"])
print("theta 3: %7.4f"% theta3, "expected: %7.4f"% current_test_set["theta3_rviz"])
print("theta 4: %7.4f"% theta4, "expected: %7.4f"% current_test_set["theta4_rviz"])
print("theta 5: %7.4f"% theta5, "expected: %7.4f"% current_test_set["theta5_rviz"])
print("theta 6: %7.4f"% theta6, "expected: %7.4f"% current_test_set["theta6_rviz"],"\n")
print("--------------- final matrices ---------------")
theta_values = { q1: theta1, q2: theta2, q3: theta3, q4: theta4, q5: theta5, q6: theta6 }
print("T0_G_corr: ", T0_G_corr.evalf(subs=theta_values))
print("Rrpy: ", Rrpy.evalf())

---------------- input data ----------------
px, py, pz:  2.1614  -1.4264   1.5511
r , p , y :  1.6544   0.4899   0.0624

--------------- wrist center ---------------
WC_x:  1.8945 expected:  1.8945
WC_y: -1.4430 expected: -1.4430
WC_z:  1.6937 expected:  1.6937 

--------------- joint angles ---------------
theta 1: -0.6509 expected: -0.6500
theta 2:  0.4482 expected: -0.6500
theta 3: -0.3621 expected: -0.6500
theta 4:  0.0396 expected:  0.9500
theta 5:  1.4515 expected:  0.7900
theta 6:  0.7225 expected:  0.4900 

--------------- final matrices ---------------
T0_G_corr:  Matrix([
[ 0.050275514095821,  0.980077184976624,    0.192148599190047,  1.90974271711526],
[0.0111644382669818,  0.191828370220246,   -0.981364983936571, -1.43963968405929],
[-0.998672983513915, 0.0514838602567926, -0.00129774129756247,  1.39107499471257],
[                 0,                  0,                    0,               1.0]])
Rrpy:  Matrix([
[ 0.880662586322695,    0.47318900979889,  0.0229253147626878

Project structure for running `IK_server.py`

To save some time on matrix calculations I introduced to new python classes:

*Forward_Kinematics.py*: This class is created when `IK_server.py` is started. On instantiation (`__init__`) it performs the matrix calculations needed for the transformation matrices. Its main function is to precalculate CPU-heavy operations and hand over the results of this transformation to the Inverse_Kinematics-Class. It contains the following functions you have already seen in this notebook:
- `get_gripper_correction`
- `build_transformation_matrix`

*Inverse_Kinematics.py*: This class is created once per joint-angle calculation. With the information given by the Forward_Kinematics-Class, it calculates Rrpy, the wrist center and the joint angles.Rrpy is calculated and modified completely inside this class, so get_wrist_rot_matrix does not return anything and get_wrist_pos does neither take Rrpy as input, nor does it return it. Inverse_Kinematics contains the following functions you have already seen in this notebook:
- `get_wrist_rot_matrix`
- `get_wrist_pos`
- `get_theta_123`
- `transform_to_wc`
- `get_theta_456`

SyntaxError: invalid syntax (<ipython-input-17-a128b681c636>, line 1)